# Calculating probability of chess moves

For each board score state (Winning, Neutral, Losing) to each end result (Win/Lose/Draw)

Collect move profiles:
- First move
- Number of times piece is moved until result

Final probabilities
- Prob of moving specific piece as first move for each starting state to final result
- Proportion of moves for each piece for each starting state to final result (like an overall strategy)

In [787]:
import os
import json
import zipfile

import chess

import numpy as np

### Helper functions

Calculate board score

In [788]:
def calc_score(boardState):
    piece_value = {'p': 1, 'n': 3, 'b': 3, 'r': 5, 'q': 9, 'k': 0}

    black_pieces = [piece["type"] for piece in boardState if piece["color"] == "b"]
    white_pieces = [piece["type"] for piece in boardState if piece["color"] == "w"]

    black_score = sum([piece_value[piece] for piece in black_pieces])
    white_score = sum([piece_value[piece] for piece in white_pieces])

    return black_score, white_score, black_pieces, white_pieces


Parse available pieces

In [789]:
def parse_available_pieces(boardState, isDingBlack):
    black = [piece["type"].capitalize() for piece in boardState if piece["color"] == "b"]
    white = [piece["type"].capitalize() for piece in boardState if piece["color"] == "w"]
    if isDingBlack:
        return black, white
    else:
        return white, black

Parse Move

In [790]:
def parse_move(move):
    piece_codes = ['K', 'Q', 'B', 'N', "R"]
    if move[0] not in piece_codes:
        move_piece = 'P'
    else:
        move_piece = move[0]
    
    return move_piece

Get captured piece

In [791]:
def get_captured_piece(game, moves):
    isDingBlack = game["isCarlsenBlack"]
    nextTurn = game["nextTurn"]
    isDingNextTurn = (isDingBlack and nextTurn == 'b') or (not isDingBlack and nextTurn == 'w')
    pieces = game["boardState"]
    piece_type_map = {'p': 1, 'n': 2, 'b': 3, 'r': 4, 'q': 5, 'k': 6}
    reverse_map = {v:k for k,v in piece_type_map.items()}
    
    base_board = chess.BaseBoard(board_fen=None)
    for piece in pieces:
        square = chess.parse_square(piece['square'])
        piece_type = piece_type_map[piece['type']]
        color = piece['color'] == 'w'
        piece_object = chess.Piece(piece_type, color)
        piece_name = None
        base_board.set_piece_at(square, piece_object)
    fen = base_board.board_fen()
    board = chess.Board(fen=fen)

    board.turn = nextTurn == 'w'
    for move in moves:
        board.push_san(move)
    last_move_square = board.pop().to_square
    captured_piece = board.piece_type_at(last_move_square)

    return reverse_map[captured_piece].upper()

Parse all moves

In [792]:
def parse_moves_to_capture(game, moves, isDingNextTurn):
    for i, move in enumerate(moves):
        if 'x' in move:
            moves = moves[:i+1]
            break

    odd_index = moves[::2]
    even_index = moves[1::2]
    if isDingNextTurn:
        if odd_index and 'x' in odd_index[-1]:
            captured_piece = get_captured_piece(game, moves)
        else:
            captured_piece = False
        return [parse_move(move) for move in odd_index], captured_piece
    else:
        if even_index and 'x' in even_index[-1]:
            captured_piece = get_captured_piece(game, moves)
        else:
            captured_piece = False
        return [parse_move(move) for move in even_index], captured_piece

Parse Win

In [793]:
def parse_win(game):
    isDingBlack = game["isCarlsenBlack"]
    result = game["result"]
    if result == "1/2-1/2":
        return "Draw"
    white_result = result[0]
    if (white_result == "1" and not isDingBlack) or (white_result == "0" and isDingBlack):
        return "Win"
    else:
        return "Lose"

Loading one game

In [794]:
def process_one_game(game, all_moves_winning_state, all_captures_winning_state):
    black_score, white_score, black_pieces, white_pieces = calc_score(game["boardState"])
    isDingBlack = game["isCarlsenBlack"]
    nextTurn = game["nextTurn"]
    moves = game["moves"]
    ding_score = black_score if isDingBlack else white_score
    opp_score = white_score if isDingBlack else black_score
    isDingNextTurn = (isDingBlack and nextTurn == 'b') or (not isDingBlack and nextTurn == 'w')

    if ding_score > opp_score:
        ding_position = "Winning"
    elif ding_score == opp_score:
        ding_position = "Neutral"
    else:
        ding_position = "Losing"

    available_pieces, opp_available_pieces = parse_available_pieces(game["boardState"], isDingBlack)


    if isDingNextTurn:
        first_move = parse_move(moves[0])
    else:
        if len(moves) <= 1:
            return
        first_move = parse_move(moves[1])

    all_moves, captured_piece = parse_moves_to_capture(game, moves, isDingNextTurn)
    num_moves = len(all_moves)
    values, counts = np.unique(all_moves, return_counts=True)
    counts = counts / num_moves
    ding_moves = dict(zip(values, counts.T))

    for available in available_pieces:
        all_moves_winning_state[ding_position][available + " avail"] += 1

    for moved in list(ding_moves.keys()):
        all_moves_winning_state[ding_position][moved] += ding_moves[moved]
    if captured_piece:
        for available in list(set(opp_available_pieces)):
            all_captures_winning_state[ding_position][available + " avail"] += 1
        all_captures_winning_state[ding_position][captured_piece] += 1


Initialise data dictionaries

In [795]:
data_first_move = {"K": 0, "K avail": 0, "Q": 0, "Q avail": 0, "R": 0, "R avail": 0, "N": 0, "N avail": 0, "B": 0, "B avail": 0, "P": 0, "P avail": 0}

all_moves_winning_state = {"Winning": data_first_move.copy(), "Losing": data_first_move.copy(), "Neutral": data_first_move.copy()}
all_captures_winning_state = {"Winning": data_first_move.copy(), "Losing": data_first_move.copy(), "Neutral": data_first_move.copy()}

Iterate over all json files in directory

In [796]:
directory_prefix = "./../carlsen_data/"
directory_name = "json_10.zip"
directory = os.fsencode(directory_name)
num_json = 0
    
with zipfile.ZipFile(directory_prefix + directory_name, "r") as z:
    for filename in z.namelist()[1:]:   
        with z.open(filename) as f: 
            data = f.read()  
            game = json.loads(data.decode("utf-8"))
            num_json +=1
            process_one_game(game, all_moves_winning_state, all_captures_winning_state)

print(num_json)

426


In [797]:
states = ["Winning", "Losing", "Neutral"]
# Normalise all probabilities in each (state, result) to sum to 100. If 0 -> 1
for state in states:
    all_moves_probabilities = all_moves_winning_state[state]
    all_captures_probabilities = all_captures_winning_state[state]
    num_games = all_moves_probabilities['K avail'] # Since there is always a K
    num_capture_games = all_captures_probabilities['K avail']

    final_all_moves_prob = {key: (round(value/num_games * 100) if value else 1) for (key, value) in all_moves_probabilities.items() if len(key) == 1}
    final_all_captures_prob = {key: (round(value/num_capture_games * 100) if value else 1) for (key, value) in all_captures_probabilities.items() if len(key) == 1}

    all_moves_winning_state[state] = final_all_moves_prob
    all_captures_winning_state[state] = final_all_captures_prob

### Results

In [798]:
all_captures_winning_state["Winning"]

{'K': 1, 'Q': 1, 'R': 8, 'N': 1, 'B': 1, 'P': 92}

In [799]:
for key, values in all_captures_winning_state.items():
    output = [str(v) for v in values.values()]
    print(', '.join(output) + ', // ' + key)

1, 1, 8, 1, 1, 92, // Winning
1, 4, 19, 8, 9, 60, // Losing
1, 2, 3, 3, 5, 87, // Neutral
